In [ ]:
import numpy as np
from numpy import linalg as LA
import os
import matplotlib.pyplot as plt

PATH_SOURCE = "/home/jbassham/jack/data/w92_20/outputs"

print("libraries loaded")

In [ ]:
fnam = 'lr_v1.npz'
data = np.load(os.path.join(PATH_SOURCE, fnam))
m = data['m']
fit = data['fit']
true = data['true']
print('LR Loaded')

In [ ]:
fnam = 'lr_weighted_v1.npz'
data = np.load(os.path.join(PATH_SOURCE, fnam))
m_w = data['m']
fit_w = data['fit']
true_w = data['true']

print('Weighted LR Loaded')

In [ ]:
PATH_SOURCE = "/home/jbassham/jack/data/w92_20/inputs"
fnam = 'inputs_normalized.npz'

data = np.load(os.path.join(PATH_SOURCE, fnam))

rt = data['rtn']
print('Normalized Uncertainty Loaded')

In [ ]:
fnam = 'lat_lon.npz'

data = np.load(os.path.join(PATH_SOURCE, fnam))

lat = data['lat']
lon = data['lon']

print('lat lon Loaded')

In [ ]:
fnam = f"time_today.npz"
data = np.load(os.path.join(PATH_SOURCE, fnam), allow_pickle=True)
time_today = data['time_today']
print("Time variable loaded")

years = time_today.astype('datetime64[Y]').astype(int) + 1970

test_mask  = (years >= 2019) & (years <= 2020)

test_idx  = np.where(test_mask)[0]
print("Val shape:", val_idx.shape)
print("Test shape:", test_idx.shape)

In [ ]:
plt.pcolormesh(np.real(fit_w[1,:,:]))

In [ ]:
def weighted_skill(fit, true, r):
    
    skill_weighted = 1 - np.nanvar((true - fit) / r, axis = 0) / np.nanvar(true / r, axis = 0)
    
    return skill_weighted

def skill(fit, true):
    
    skill = 1 - np.nanvar(true - fit, axis = 0) / np.nanvar(true, axis = 0)
    # Check where true variance really small!
    
    return skill

def return_plot_skill(fit, true, r, model):

    vmin = -1
    vmax = 1
    cmap = plt.cm.RdBu
    
    # Calculate skill of complex output (the ice velocity vector)
    skill_vector = skill(fit, true)
    # NOTE: Variance of complex input returned as Var(z)=E[(z−E[z])**2
    # so a real number is returned that represents the spread in both u and v components of the vector
    
    skill_u = skill(np.real(fit), np.real(true))
    skill_v = skill(np.imag(fit), np.imag(true))
    
    data = [skill_vector, skill_u, skill_v]
    titles = ['vector', 'u', 'v']
    
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(12, 9), constrained_layout=True)
    
    for i in range(3):
        pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin=vmin, vmax=vmax)
        fig.colorbar(pcm, ax=axs[i], label='Skill')
        axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')
        
    fig.suptitle(f'Skill {model}')
    plt.show()
    
    weighted_skill_vector = weighted_skill(fit, true, r)
    weighted_skill_u = weighted_skill(np.real(fit), np.real(true), r)
    weighted_skill_v = weighted_skill(np.imag(fit), np.imag(true), r)
    
    data = [weighted_skill_vector, weighted_skill_u, weighted_skill_v]
    
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(12, 9), constrained_layout=True)
    
    for i in range(3):
        pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin=vmin, vmax=vmax)
        fig.colorbar(pcm, ax=axs[i], label='Weighted Skill')
        axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')
        
    fig.suptitle(f'Weighted Skill {model}')
    plt.show()
    
    # var_true_vector = np.nanvar(true, axis = 0)
    # var_true_u = np.nanvar(np.real(true), axis = 0)
    # var_true_v = np.nanvar(np.imag(true), axis = 0)
    
    # data = [var_true_vector, var_true_u, var_true_v]
    # titles = [r'vector', r'u', r'v']
    
    # fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(12, 9), constrained_layout=True)
    
    # for i in range(3):
    #     pcm = axs[i].pcolormesh(lon, lat, data[i])
    #     fig.colorbar(pcm, ax=axs[i], label='')
    #     axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')
        
    # fig.suptitle(f'Variance {model}')
    # plt.show()
    
    return skill_vector, skill_u, skill_v, weighted_skill_vector, weighted_skill_u, weighted_skill_v

In [ ]:
wlr_skill_z, wlr_skill_u, wlr_skill_v, wlr_w_skill_z, wlr_w_skill_u, wlr_w_skill_v = return_plot_skill(fit_w, true_w, rt, 'Weighted LR')



In [ ]:
lr_skill_z, lr_skill_u, lr_skill_v, lr_w_skill_z, lr_w_skill_u, lr_w_skill_v = return_plot_skill(fit, true, rt, 'Unweighted LR')



In [ ]:
del_skill_z = wlr_w_skill_z - lr_w_skill_z
del_skill_u = wlr_w_skill_u - lr_w_skill_u
del_skill_v = wlr_w_skill_v - lr_w_skill_v

In [ ]:
data = [del_skill_z, del_skill_u, del_skill_v]
titles = ["'z' vector", "'u' zonal", "'v' meridional"]

vmin = -1
vmax = 1
cmap = plt.cm.RdBu

fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(12, 9), constrained_layout=True)

for i in range(3):
    pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin = vmin, vmax = vmax)
    fig.colorbar(pcm, ax=axs[i], label=r'$\Delta_{skill}$')
    axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')

fig.suptitle(r'$\Delta_{skill}$ (Weighted LR, LR)', fontweight = 'bold')

plt.show()


In [ ]:
data = [del_skill_z, del_skill_u, del_skill_v]
titles = ["'z' vector", "'u' zonal", "'v' meridional"]

vmin = -0.1
vmax = 0.1
cmap = plt.cm.RdBu

fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(12, 9), constrained_layout=True)

for i in range(3):
    pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin = vmin, vmax = vmax)
    fig.colorbar(pcm, ax=axs[i], label=r'$\Delta_{weighted_skill}$')
    axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')

fig.suptitle(r'$\Delta_{weighted\ skill}$ (Weighted LR, LR)', fontweight = 'bold')


plt.show()

In [ ]:
PATH_SOURCE = "/home/jbassham/jack/SIOC209/outputs"

In [ ]:
fnam = "preds_CNNtfv1.npz"
data = data = np.load(os.path.join(PATH_SOURCE, fnam))

y_pred_tf = data['y_pred']
y_pred_tf = data['y_true']

In [ ]:
fnam = "preds_CNNtorchv1.npz"
data = data = np.load(os.path.join(PATH_SOURCE, fnam))

y_pred_tor1 = data['y_pred']
y_pred_tor1 = data['y_true']

In [ ]:
fnam = "preds_CNNtorchv2Weighted.npz"
data = data = np.load(os.path.join(PATH_SOURCE, fnam))

y_pred_torw = data['y_pred']
y_pred_torw = data['y_true']

In [ ]:
def tf_return_plot_skill(fit, true, r, lat, lon, test_idx, model_str):

    vmin = -1
    vmax = 1
    cmap = plt.cm.RdBu
    
    # Calculate skill of complex output (the ice velocity vector)
    # NOTE: Variance of complex input returned as Var(z)=E[(z−E[z])**2
    # so a real number is returned that represents the spread in both u and v components of the vector
    
    skill_u = skill(fit[:,:,:,0], true[:,:,:,0])
    skill_v = skill(fit[:,:,:,1], true[:,:,:,1])
    
    data = [skill_u, skill_v]
    nd = len(data)
    titles = ['zonal velocity', 'meridional velocity']
    
    fig, axs = plt.subplots(nrows=nd, ncols=1, figsize=(12, nd*3), constrained_layout=True)
    
    for i in range(nd):
        pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin=vmin, vmax=vmax)
        fig.colorbar(pcm, ax=axs[i], label='Skill')
        axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')
        
    fig.suptitle(f'Skill {model_str}')

    # # Save plot
    # fnam = f'{model_str}_skill.jpeg'
    # plt.savefig(os.path.join(PATH_DEST, fnam), dpi=300, format='jpeg')
    # plt.show()
    
    r_test = r[test_idx]
    
    weighted_skill_u = weighted_skill(fit[:,:,:,0], true[:,:,:,0], r_test)
    weighted_skill_v = weighted_skill(fit[:,:,:,1], true[:,:,:,1], r_test)
    
    data = [weighted_skill_u, weighted_skill_v]
    
    fig, axs = plt.subplots(nrows=nd, ncols=1, figsize=(12, nd*3), constrained_layout=True)
    
    for i in range(len(data)):
        pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin=vmin, vmax=vmax)
        fig.colorbar(pcm, ax=axs[i], label='Weighted Skill')
        axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')
        
    fig.suptitle(f'Weighted Skill {model_str}')

    # # Save plot
    # fnam = f'{model_str}_weighted_skill.jpeg'
    # plt.savefig(os.path.join(PATH_DEST, fnam), dpi=300, format='jpeg')
    
    plt.show()
    
    return skill_u, skill_v, weighted_skill_u, weighted_skill_v

In [ ]:
def tor_return_plot_skill(fit, true, r, lon, lat, test_idx, model_str):

    vmin = -1
    vmax = 1
    cmap = plt.cm.RdBu
    
    skill_u = skill(fit[:,0,:,:], true[:,0,:,:])
    skill_v = skill(fit[:,1,:,:], true[:,1,:,:])
    
    data = [skill_u, skill_v]
    nd = len(data)
    titles = ['zonal velocity', 'meridional velocity']
    
    fig, axs = plt.subplots(nrows=nd, ncols=1, figsize=(12, nd*3), constrained_layout=True)
    
    for i in range(nd):
        pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin=vmin, vmax=vmax)
        fig.colorbar(pcm, ax=axs[i], label='Skill')
        axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')
        
    fig.suptitle(f'Skill {model_str}', fontweight = 'bold')

    # # Save plot
    # fnam = f'{model_str}_skill.jpeg'
    # plt.savefig(os.path.join(PATH_DEST, fnam), dpi=300, format='jpeg')
    
    plt.show()
    
    r_test = r[test_idx]
    
    weighted_skill_u = weighted_skill(fit[:,0,:,:], true[:,0,:,:], r_test)
    weighted_skill_v = weighted_skill(fit[:,1,:,:], true[:,1,:,:], r_test)
    
    data = [weighted_skill_u, weighted_skill_v]
    
    fig, axs = plt.subplots(nrows=nd, ncols=1, figsize=(12, nd*3), constrained_layout=True)
    
    for i in range(len(data)):
        pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin=vmin, vmax=vmax)
        fig.colorbar(pcm, ax=axs[i], label='Weighted Skill')
        axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')
        
    fig.suptitle(f'Weighted Skill {model_str}', fontweight = 'bold')

    # # Save plot
    # fnam = f'{model_str}_weighted_skill.jpeg'
    # plt.savefig(os.path.join(PATH_DEST, fnam), dpi=300, format='jpeg')
    
    plt.show()
    
    return skill_u, skill_v, weighted_skill_u, weighted_skill_v

In [ ]:
tf_s_u, tf_s_v, tf_ws_u, tf_ws_v = tf_return_plot_skill(y_pred_tf, y_true_tf, rt, lat, lon, test_idx, 'CNN tensorflow')

In [ ]:
tor1_s_u, tor1_s_v, tor1_ws_u, tor1_ws_v = tor_return_plot_skill(y_pred_tor1, y_true_tor1, rt, lat, lon, test_idx, 'CNN pytorch')

In [ ]:
torw_s_u, torw_s_v, torw_ws_u, torw_ws_v = tor_return_plot_skill(y_pred_torw, y_true_torw, rt, lat, lon, test_idx, 'Weighted CNN pytorch')

In [ ]:
del_tortf_u = tor1_s_u - tf_s_u
del_tortf_v = tor1_s_v - tf_s_v

In [ ]:
data = [del_tortf_u, del_tortf_v]
titles = ["'u' zonal", "'v' meridional"]

vmin = -0.1
vmax = 0.1
cmap = plt.cm.RdBu

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 6), constrained_layout=True)

for i in range(3):
    pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin = vmin, vmax = vmax)
    fig.colorbar(pcm, ax=axs[i], label=r'$\Delta_{weighted_skill}$')
    axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')

fig.suptitle(r'$\Delta_{weighted\ skill}$ (PyTorch, Tensorflow)', fontweight = 'bold')


plt.show()

In [ ]:
del_wtor_u = torw_s_u - tor1_s_u
del_wtor_v = torw_s_v - tor1_s_v

In [ ]:
data = [del_wtor_u, del_wtor_v]
titles = ["'u' zonal", "'v' meridional"]

vmin = -0.1
vmax = 0.1
cmap = plt.cm.RdBu

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 6), constrained_layout=True)

for i in range(3):
    pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin = vmin, vmax = vmax)
    fig.colorbar(pcm, ax=axs[i], label=r'$\Delta_{weighted_skill}$')
    axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')

fig.suptitle(r'$\Delta_{weighted\ skill}$ (Pytorch Weighted CNN, Pytorch CNN)', fontweight = 'bold')


plt.show()

In [ ]:
del_wtorlr_u = torw_s_u - wlr_skill_u
del_wtorlr_v = torw_s_v - wlr_skill_v

data = [del_wtorlr_u, del_wtorlr_v]
titles = ["'u' zonal", "'v' meridional"]

vmin = -0.1
vmax = 0.1
cmap = plt.cm.RdBu

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 6), constrained_layout=True)

for i in range(3):
    pcm = axs[i].pcolormesh(lon, lat, data[i], cmap=cmap, vmin = vmin, vmax = vmax)
    fig.colorbar(pcm, ax=axs[i], label=r'$\Delta_{weighted_skill}$')
    axs[i].set(title=titles[i], xlabel='Lon', ylabel='Lat')

fig.suptitle(r'$\Delta_{weighted\ skill}$ (Pytorch Weighted CNN, Weighted LR)', fontweight = 'bold')


plt.show()